# Relative Annual Measures: What happened after program enrollment?

Note to users: please read the instructions file in this folder (00_instructions) before using this Jupyter Notebook file.

## SQL Database Connection

This section loads needed packages and connects the IPython Jupyter Notebook to the SQL database. If you are running this code in your own environment, remember to modify the SQL connection string to route the notebook to your own SQL server and database (see the 00_instructions file in this folder for more information). Our code uses the SQLALchemy Python package to interface between python and SQL languages, and uses Jupyter SQL ‘magic’ functions to make the code more concise.


In [10]:
# load sqlalchemy package to interface between Python and SQL databases
import sqlalchemy

# Replace the SQL connection string below (in quotation marks) with your own SQL connection information to run the program
connection_string = "mssql+pyodbc://@TDI"

# Create the engine connecting to the database server
sqlalchemy.create_engine(connection_string)

# Load the ipython-sql library to use Jupyter 'magic' functions, which make your code more concise
%load_ext sql

# Connect to the database server
%sql $connection_string

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Purpose:

In the previous notebook (06_relative_measures_qtr), we created quarterly employment and earning measures that were relative to the program enrollment quarter. The purpose of this notebook is similar to the previous one, except here we create annual relative measures. This code will create measures using the data for the 4 quarters (relative year) after program enrollment. Reminder that these are called "relative" measures because they are relative to the enrollment year, unlike the simple calendar measures we created in the 05_calendar_measures_annual notebook. We create annual relative measures that correspond with the annual calendar measures we created previously: any employment, average quarterly employment, and annual earnings. We also create measures that can reflect employment stability or advancement, including whether someone earned more than $3500 in any quarter over the course of a year, or, for people with enough follow-up data, whether their earnings increased over time.

### Starting Point

Our source file for these relative measures already has a record for every possible quarter in our follow-up period. Each record has information about the earnings reported, a yes/no indicator of employment (0/1), and the number of employers who reported earnings for the person during the quarter. We will keep and use only the quarter just before program enrollment, the quarter of program enrollment, and 4 quarters after program enrollment. We have already created a column (RelativeQTR) that indicates how close each quarter is to the client's program enrollment, but we need to add a column that groups four quarters together because we want "relative yearly" measures.

Let's take a look at the file that we will use as input and view all records for the first client (17 total) and the first record for the second client. Notice:

1.	The first client below enrolled in the program in quarter 2 of 2017 (06/01/2017)
2.	Earnings reported in quarter 1 of 2017 (1 quarter prior to enrollment) are assigned the RelativeQTR value of -1
3.	Earnings reported in quarter 2 of 2017 (the first quarter enrolled) are assigned the RelativeQTR value of 0. This client has data for 15 quarters post-program enrollment.
4.	The second client below enrolled in the program in quarter 1 of 2017
5. Quarter 1 of 2017 is the first quarter of UI wage data we received. Therefore, we have no data for the quarter before program enrollment and no record with RelativeQTR value -1 for the second client.

In [11]:
%%sql
CREATE or ALTER VIEW RelYearV as
SELECT *,
(((
    (CASE WHEN RelativeQTR >0 then RelativeQTR END)
  -1)/4)+1) as RelYear -- add column that indicates relative year the quarter belongs to
FROM UIQuarterlyMeasuresV;

SELECT TOP 18 * -- print records for first case
FROM RelYearV
order by SSN, YR_QTR;

 * mssql+pyodbc://@TDI
Done.
Done.


SSN,ProgStart,ProgEnd,YR_QTR,EarnQTR,RelativeQTR,QTR_Earnings,QTR_EMPLOYED,QTR_NUMEMPLOYERS,RelYear
100000000,2017-06-01,2017-11-28,2017Q1,2017-01-01,-1,9214,1,1,None
100000000,2017-06-01,2017-11-28,2017Q2,2017-04-01,0,8561,1,1,None
100000000,2017-06-01,2017-11-28,2017Q3,2017-07-01,1,12550,1,1,1
100000000,2017-06-01,2017-11-28,2017Q4,2017-10-01,2,0,0,0,1
100000000,2017-06-01,2017-11-28,2018Q1,2018-01-01,3,0,0,0,1
100000000,2017-06-01,2017-11-28,2018Q2,2018-04-01,4,0,0,0,1
100000000,2017-06-01,2017-11-28,2018Q3,2018-07-01,5,0,0,0,2
100000000,2017-06-01,2017-11-28,2018Q4,2018-10-01,6,0,0,0,2
100000000,2017-06-01,2017-11-28,2019Q1,2019-01-01,7,3804,1,1,2
100000000,2017-06-01,2017-11-28,2019Q2,2019-04-01,8,17082,1,1,2


###  First, a word about data coverage and missing data

Remember that the last quarter of UI wage data that we received is Q1 2021. We do not have data for Q2 2021 or later for any client. Let's take a look at the data for the person with the latest enrollment date in the sample definition file. Notice:

1.	This person started in the program in December 2020.
2.	Because of this person’s late start relative to the wage data coverage period, the earnings records cover mostly periods prior to their program enrollment: Beyond that, we have wage information only for the quarter this client started in our program and 1 quarter after program start.
3.	Therefore, any annual measures we create for this person, including periods after relative quarter 1,  should be missing and excluded from analysis on annual follow-up periods because the data are incomplete.

In [12]:
%%sql
SELECT *
FROM RelYearV
where ProgStart=  -- select the client with a start date equal to the last start date on the file
    (select MAX(ProgStart)
     FROM RelYearV) -- finding the last program start date on the file
;

 * mssql+pyodbc://@TDI
Done.


SSN,ProgStart,ProgEnd,YR_QTR,EarnQTR,RelativeQTR,QTR_Earnings,QTR_EMPLOYED,QTR_NUMEMPLOYERS,RelYear
919951016,2020-12-26,None,2017Q1,2017-01-01,-15,0,0,0,None
919951016,2020-12-26,None,2017Q2,2017-04-01,-14,841,1,1,None
919951016,2020-12-26,None,2017Q3,2017-07-01,-13,9851,1,1,None
919951016,2020-12-26,None,2017Q4,2017-10-01,-12,0,0,0,None
919951016,2020-12-26,None,2018Q1,2018-01-01,-11,6340,1,1,None
919951016,2020-12-26,None,2018Q2,2018-04-01,-10,5243,1,1,None
919951016,2020-12-26,None,2018Q3,2018-07-01,-9,0,0,0,None
919951016,2020-12-26,None,2018Q4,2018-10-01,-8,10271,1,1,None
919951016,2020-12-26,None,2019Q1,2019-01-01,-7,0,0,0,None
919951016,2020-12-26,None,2019Q2,2019-04-01,-6,223,1,1,None


#### Careful: unless steps are taken, summary measures will be calculated for all cases with ANY data available

1.	The code below demonstrates what can happen when the summary measure, total earnings in the year after program enrollment, is created for the last client enrolled, who does not have data for all 4 quarters.
2.	We only keep records that are for relative quarters 1 through 4 (the 4 quarters after the enrollment quarter).
3.	We sum earnings for those quarters by SSN.
4.	Notice that total earnings for year 1 after enrollment is calculated even though data for relative quarters 2-4 are not yet available for this client. 

In [13]:
%%sql
SELECT 
SSN, RelYear,

   SUM(QTR_Earnings) AS EARNy  -- total earning in year 1 after start

FROM RelYearV /* using quarter table view */

where RelativeQTR >= 1 and -- we only want to include records for the quarters AFTER the program enrollment quarter
    ProgStart=  (select MAX(ProgStart)
     FROM RelYearV)  -- we just want look at the last client enrolled as an example of missing data
    
GROUP BY SSN,RelYear /* group by is SQL way of reducing multiple record to 1 record per person */
;

 * mssql+pyodbc://@TDI
Done.


SSN,RelYear,EARNy
919951016,1,0


### Controlling the calculations by checking for the maximum number of quarters of data the client has 

There are many ways to manage calculations for those who don't have enough data. If you want every client to have a record in your result set, even though it will have NULL values, you can do this by testing that the count of the number of records is 4 AS you are coding your summary calculations (CASE WHEN COUNT(\*)=4 THEN 1 END). If the count is not 4 then the expression returns NULL. This approach is demonstrated below.

In [14]:
%%sql
CREATE or ALTER VIEW RelYearSumV as
SELECT SSN, RelYear,

/* relative year measures*/
-- employment measures
MAX(QTR_Employed)*(CASE WHEN COUNT(*)=4 THEN 1 END) AS EMPy, -- was employed sometime in year 1
SUM(QTR_Employed)*(CASE WHEN COUNT(*)=4 THEN 1 END) AS KEMPy, -- number of quarter employed
AVG(CAST(QTR_Employed AS DECIMAL(4,2)))*(CASE WHEN COUNT(*)=4 THEN 1 END) AS PerQEMPy, -- Percent of quarter employed
    
-- earnings measures
SUM(QTR_Earnings)*(CASE WHEN COUNT(*)=4 THEN 1 END) AS EARNy, -- total earning in year 1 after start
SUM(CASE WHEN QTR_Earnings >= 3500 THEN 1 ELSE 0 END)*(CASE WHEN COUNT(*)=4 THEN 1 END)
AS KQGT3500y, -- Number of QTRs w/earning ge 3500

(SUM(CASE WHEN QTR_Earnings >= 3500 THEN 1.0 ELSE 0 END)/4)*(CASE WHEN COUNT(*)=4 THEN 1 END)
AS PerQGT3500y, -- Percent of QTRs w/earning ge 3500

-- creating  measures of how much data relative data a client has
COUNT(*) AS NumQTR,
MAX( RelativeQTR) AS LastRelQTR

FROM RelYearV /* using quarter table view */
where RelativeQTR >=1
GROUP BY SSN, RelYear; /* group by is SQL way of reducing multiple record to 1 record per person */

SELECT TOP 8 *
FROM RelYearSumV
ORDER BY SSN, RelYear;

 * mssql+pyodbc://@TDI
Done.
Done.


SSN,RelYear,EMPy,KEMPy,PerQEMPy,EARNy,KQGT3500y,PerQGT3500y,NumQTR,LastRelQTR
100000000,1,1,1,0.250000,12550,1,0.250000,4,4
100000000,2,1,2,0.500000,20886,2,0.500000,4,8
100000000,3,1,1,0.250000,1881,0,0.000000,4,12
100000000,4,None,None,None,None,None,None,3,15
100900056,1,1,2,0.500000,15363,2,0.500000,4,4
100900056,2,1,1,0.250000,7582,1,0.250000,4,8
100900056,3,1,3,0.750000,13006,2,0.500000,4,12
100900056,4,1,1,0.250000,8172,1,0.250000,4,16


### FLATTEN FILE WITH GROUP BY and CASE: Relative Year 1 and Year 2 Measures

1.	We reduce the multiple records (displayed above) per person 1 record per person by using the GROUP BY statement. The GROUP BY statement requires the use of an aggregate function – here so we are using SUM.

2.	The CASE statement determines if the record is for the year of interest If it is, it assigns the value to the column. If not, the record is skipped.

3.	We use the relative quarter value (RelativeQTR) in the WHERE statement to select the rows that we want included in the calculation. As mentioned above,  the quarter that the client started in the program is considered relative quarter 0, and is not included in the post-program period, which starts with relative quarter 1.

In [15]:
%%sql
CREATE or ALTER VIEW FlatUIV as
SELECT SSN,

/* relative year measures*/
-- employment measures
SUM(CASE RelYear WHEN 1 THEN EMPy END) AS EMPy1, -- was employed sometime in year 1
SUM(CASE RelYear WHEN 1 THEN KEMPy END) AS KEMPy1, -- number of quarter employed
SUM(CASE RelYear WHEN 1 THEN PerQEMPy END) AS PQEMPy1, -- Percent of quarter employed
    
-- earnings measures
SUM(CASE RelYear WHEN 1 THEN EARNy END) AS EARNy1, -- total earning in year 1 after start
SUM(CASE RelYear WHEN 1 THEN KQGT3500y END) AS KQGT3500y1, -- Number of QTRs w/earning ge 3500
SUM(CASE RelYear WHEN 1 THEN PerQGT3500y END) AS PQGT3500y1, -- Number of QTRs w/earning ge 3500


/* relative year measures*/
-- employment measures
SUM(CASE RelYear WHEN 2 THEN EMPy END) AS EMPy2, -- was employed sometime in year 2
SUM(CASE RelYear WHEN 2 THEN KEMPy END) AS KEMPy2, -- number of quarter employed
SUM(CASE RelYear WHEN 2 THEN PerQEMPy END) AS PQEMPy2, -- Percent of quarter employed
    
-- earnings measures
SUM(CASE RelYear WHEN 2 THEN EARNy END) AS EARNy2, -- total earning in year 2 after start
SUM(CASE RelYear WHEN 2 THEN KQGT3500y END) AS KQGT3500y2, -- Number of QTRs w/earning ge 3500
SUM(CASE RelYear WHEN 2 THEN PerQGT3500y END) AS PQGT3500y2, -- Number of QTRs w/earning ge 3500

-- keep track of how much quarterly data each person had
MAX(LastRelQTR) AS LastQTR

FROM RelYearSumV
GROUP BY SSN;/* group by is SQL way of reducing multiple record to 1 record per person */

SELECT TOP 10 *
FROM FlatUIV
ORDER BY SSN;


 * mssql+pyodbc://@TDI
Done.
Done.


SSN,EMPy1,KEMPy1,PQEMPy1,EARNy1,KQGT3500y1,PQGT3500y1,EMPy2,KEMPy2,PQEMPy2,EARNy2,KQGT3500y2,PQGT3500y2,LastQTR
100000000,1,1,0.250000,12550,1,0.250000,1,2,0.500000,20886,2,0.500000,15
100900056,1,2,0.500000,15363,2,0.500000,1,1,0.250000,7582,1,0.250000,16
101800112,1,3,0.750000,46458,3,0.750000,1,2,0.500000,11535,1,0.250000,14
102700168,1,1,0.250000,1324,0,0.000000,1,2,0.500000,12074,1,0.250000,15
103600224,1,2,0.500000,14720,2,0.500000,1,2,0.500000,5117,1,0.250000,15
104500280,1,1,0.250000,9720,1,0.250000,1,3,0.750000,15078,1,0.250000,15
105400336,1,1,0.250000,8254,1,0.250000,1,2,0.500000,19027,2,0.500000,15
106300392,1,4,1.000000,27910,3,0.750000,1,3,0.750000,24574,3,0.750000,15
107200448,1,2,0.500000,17414,2,0.500000,1,2,0.500000,8565,1,0.250000,16
108100504,1,3,0.750000,21789,2,0.500000,1,3,0.750000,12107,1,0.250000,14


### Check the year 2 calculations for clients with less than 2 years (or 8 quarters) of follow-up data
The check below is used to demonstrate that clients without the full year of data for relative year 2 are set to NULL.

In [16]:
%%sql
SELECT TOP 5 * 
FROM FlatUIV /* using quarter table view */
WHERE LastQTR < 8 -- LOOKING AT CLIENTS WITH TRUNCATED DATA
ORDER BY SSN
;

 * mssql+pyodbc://@TDI
Done.


SSN,EMPy1,KEMPy1,PQEMPy1,EARNy1,KQGT3500y1,PQGT3500y1,EMPy2,KEMPy2,PQEMPy2,EARNy2,KQGT3500y2,PQGT3500y2,LastQTR
710237968,1,2,0.500000,7550,1,0.250000,None,None,None,None,None,None,7
712038080,0,0,0.000000,0,0,0.000000,None,None,None,None,None,None,7
713838192,1,2,0.500000,22656,2,0.500000,None,None,None,None,None,None,7
716538360,1,1,0.250000,1392,0,0.000000,None,None,None,None,None,None,7
718338472,1,1,0.250000,10751,1,0.250000,None,None,None,None,None,None,7


## Automating the code above using Dynamic SQL

The code presented above has been written manually for demonstration purposes. However, many of the measures we have created are yearly measures that are coded the same way for each time point, and you may want to follow employment trends for people over many years. To create 5 measures for a 4 year follow-up period would take many lines of logic and summation code. This section demonstrates how to generate and run automated SQL queries and create relative annual measures dynamically.

This type of SQL coding is known as **Dynamic SQL**. The result will be query language that can be run (or executed). So, we are not looking to generate SQL result sets below; rather we use this code to generate SQL query code that can then be run.

**Note about Dynamic SQL:**

*Jupyter Notebooks, the file type we are using to share this code, does not support dynamic SQL code. The code below therefore produces errors when it is executed here. To use this code, you should copy and paste it into your respective SQL Server software.*

Review the Create View code above that created 2 years of earning and employment measures but suppose you wanted to create these measures for all the years we had data for. Notice that all the lines above are similar but

1.	The value that the WHEN clause tests for changes for every relative year: 1, 2, …
2.	The name of the measure being created changes for every relative year: EARNy1, EMPy1, EARNy2, EMPy2 ...

Below, the distinct values of the relative year variable will be stored in an iterative variable (@i) to drive the creation of our SQL query. By using this iterative variable, the only thing that changes in each line is the value of the relative year. The value of the year is tested in the WHEN clause and is added as a suffix at the end of the column names.

In [17]:
%%sql

DECLARE @CMD NVARCHAR(MAX) ; -- create a temporary variable to store our automated sql code
SELECT @CMD = 
'SELECT SSN,'; -- store the start of the query code

DECLARE @i INT=1; -- create a variable for looping

WHILE @i <=4
BEGIN

SELECT @CMD=@CMD + ' 

SUM(CASE RelYear WHEN ' + cast(@i as nvarchar(10)) + ' THEN EMPy END) AS EMPy' + cast(@i as nvarchar(10)) + ',
SUM(CASE RelYear WHEN ' + cast(@i as nvarchar(10)) + ' THEN KEMPy END) AS KEMPy' + cast(@i as nvarchar(10))+ ',
SUM(CASE RelYear WHEN ' + cast(@i as nvarchar(10)) + ' THEN PerQEMPy END) AS PerQEMPy' + cast(@i as nvarchar(10))+ ',
SUM(CASE RelYear WHEN ' + cast(@i as nvarchar(10)) + ' THEN EARNy END) AS EARNy' + cast(@i as nvarchar(10))+ ',
SUM(CASE RelYear WHEN ' + cast(@i as nvarchar(10)) + ' THEN KQGT3500y END) AS KQGT3500y' + cast(@i as nvarchar(10))+ ','


SELECT @i= @i+1;
END;


SELECT @CMD=@CMD+ '
MAX(LastRelQTR) AS LastRelQTR
FROM  RelYearSumV
GROUP BY SSN;'
; -- append the end of the query code

PRINT @CMD; -- print the query code we generated
EXEC sp_executesql @CMD; -- execute the code we generated


 * mssql+pyodbc://@TDI
Done.
(pyodbc.ProgrammingError) ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Must declare the scalar variable "@CMD". (137) (SQLExecDirectW)')
[SQL: SELECT @CMD = 
'SELECT SSN,'; -- store the start of the query code]
(Background on this error at: http://sqlalche.me/e/14/f405)


#### For reference, the dynamic SQL coding above generates and executes the SQL Query code below (remember, this code will produce errors when run in Jupyter Notebooks. See the note above about dynamic SQL):

In [18]:
SELECT SSN, 

SUM(CASE RelYear WHEN 1 THEN EMPy END) AS EMPy1,
SUM(CASE RelYear WHEN 1 THEN KEMPy END) AS KEMPy1,
SUM(CASE RelYear WHEN 1 THEN PerQEMPy END) AS PerQEMPy1,
SUM(CASE RelYear WHEN 1 THEN EARNy END) AS EARNy1,
SUM(CASE RelYear WHEN 1 THEN KQGT3500y END) AS KQGT3500y1, 

SUM(CASE RelYear WHEN 2 THEN EMPy END) AS EMPy2,
SUM(CASE RelYear WHEN 2 THEN KEMPy END) AS KEMPy2,
SUM(CASE RelYear WHEN 2 THEN PerQEMPy END) AS PerQEMPy2,
SUM(CASE RelYear WHEN 2 THEN EARNy END) AS EARNy2,
SUM(CASE RelYear WHEN 2 THEN KQGT3500y END) AS KQGT3500y2, 

SUM(CASE RelYear WHEN 3 THEN EMPy END) AS EMPy3,
SUM(CASE RelYear WHEN 3 THEN KEMPy END) AS KEMPy3,
SUM(CASE RelYear WHEN 3 THEN PerQEMPy END) AS PerQEMPy3,
SUM(CASE RelYear WHEN 3 THEN EARNy END) AS EARNy3,
SUM(CASE RelYear WHEN 3 THEN KQGT3500y END) AS KQGT3500y3, 

SUM(CASE RelYear WHEN 4 THEN EMPy END) AS EMPy4,
SUM(CASE RelYear WHEN 4 THEN KEMPy END) AS KEMPy4,
SUM(CASE RelYear WHEN 4 THEN PerQEMPy END) AS PerQEMPy4,
SUM(CASE RelYear WHEN 4 THEN EARNy END) AS EARNy4,
SUM(CASE RelYear WHEN 4 THEN KQGT3500y END) AS KQGT3500y4,
MAX(LastRelQTR) AS LastRelQTR
FROM  RelYearSumV
GROUP BY SSN;


SyntaxError: invalid syntax (<ipython-input-18-72f088ec16c5>, line 1)

## Any Change in Earnings Over Time?

We are often interested in comparing a client's quarterly earnings in the first year in the program to later years following program participation. A common approach is to look at the change in maximum quarterly earnings after four years of program participation. Below, we find the highest (or maximum) quarterly earnings during the first year of the program and compare it to the highest quarterly earnings during the fourth year. We create a variable storing the difference between the earnings (ChangeY1ToY4) and a binary variable for whether the difference is greater than $250 (ChngY1Y4GE250).

In [19]:
%%sql
SELECT TOP 10
SSN,

/* relative quarter measures */
SUM(CASE RelativeQTR WHEN 1 THEN QTR_Earnings END) AS EARN1, -- displaying year 1 quarterly earnings
SUM(CASE RelativeQTR WHEN 2 THEN QTR_Earnings END) AS EARN2,
SUM(CASE RelativeQTR WHEN 3 THEN QTR_Earnings END) AS EARN3,
SUM(CASE RelativeQTR WHEN 4 THEN QTR_Earnings END) AS EARN4,
SUM(CASE RelativeQTR WHEN 13 THEN QTR_Earnings END) AS EARN13, -- displaying year 4 quarterly earnings
SUM(CASE RelativeQTR WHEN 14 THEN QTR_Earnings END) AS EARN14, 
SUM(CASE RelativeQTR WHEN 15 THEN QTR_Earnings END) AS EARN15,
SUM(CASE RelativeQTR WHEN 16 THEN QTR_Earnings END) AS EARN16,

MAX(CASE WHEN RelativeQTR in(1,2,3,4) THEN QTR_Earnings END) AS MAXEARNy1, -- Max quarterly earnings in year 1 
MAX(CASE WHEN RelativeQTR in(13,14,15,16) THEN QTR_Earnings END) AS MAXEARNy4, -- Max quarterly earnings in year 4

/* calculate the difference between year 1 max and year 4 max*/
(
MAX(CASE WHEN RelativeQTR in(13,14,15,16) THEN QTR_Earnings END) -
MAX(CASE WHEN RelativeQTR in(1,2,3,4) THEN QTR_Earnings END)
)  AS ChangeY1ToY4,

case when(
MAX(CASE WHEN RelativeQTR in(13,14,15,16) THEN QTR_Earnings END) -
MAX(CASE WHEN RelativeQTR in(1,2,3,4) THEN QTR_Earnings END)
) >= 250 then 1 else 0 end AS ChngY1Y4GE250, -- is the change $250 or more

MAX(RelativeQTR) AS LastQTR -- check each client has enough data to be included in these measures

FROM UIQuarterlyMeasuresV /* using quarter table view */
GROUP BY SSN /* group by is SQL way of reducing multiple record to 1 record per person */
HAVING MAX(RelativeQTR) >= 16 -- client needs at least 4 years of data to be appropriate for these calculations
ORDER BY SSN
;

 * mssql+pyodbc://@TDI
Done.


SSN,EARN1,EARN2,EARN3,EARN4,EARN13,EARN14,EARN15,EARN16,MAXEARNy1,MAXEARNy4,ChangeY1ToY4,ChngY1Y4GE250,LastQTR
100900056,4371,10992,0,0,0,8172,0,0,10992,8172,-2820,0,16
107200448,0,8574,0,8840,3482,27033,3350,16783,8840,27033,18193,1,16
113500840,0,0,703,6523,0,6331,0,7971,6523,7971,1448,1,16
119801232,3543,7184,8780,6773,0,0,1251,3121,8780,3121,-5659,0,16
126101624,30137,0,0,18499,0,0,0,0,30137,0,-30137,0,16
131501960,0,0,0,0,6871,0,3573,0,0,6871,6871,1,16
137802352,783,20722,9053,19942,0,0,0,0,20722,0,-20722,0,16
144102744,0,7350,0,0,12343,3082,0,1232,7350,12343,4993,1,16
150403136,0,0,0,2033,1552,0,0,0,2033,1552,-481,0,16
156703528,7801,0,281,0,0,0,7713,0,7801,7713,-88,0,16
